In [217]:
import csv

#Reading Csv file

csv_file= "Road_Distance.csv"

with open(csv_file, mode='r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        print(row)

['Distance in Kilometres', 'Ahmedabad', 'Bangalore', 'Bhubaneshwar', 'Bombay', 'Calcutta', 'Chandigarh', 'Cochin', 'Delhi', 'Hyderabad', 'Indore', 'Jaipur', 'Kanpur', 'Lucknow', 'Madras', 'Nagpur', 'Nasik', 'Panjim', 'Patna', 'Pondicherry', 'Pune']
['Agartala', '3305', '3824', '2286', '3593', '1863', '2998', '4304', '2708', '3330', '2891', '2801', '2281', '2252', '3493', '2696', '3365', '3507', '1681', '3661', '3442']
['Agra', '878', '1848', '1578', '1202', '1300', '448', '2278', '200', '1246', '591', '230', '290', '369', '2048', '770', '1005', '1715', '885', '2210', '1214']
['Ahmedabad', '-', '1490', '1697', '552', '2068', '1157', '1845', '911', '1436', '442', '648', '1168', '1247', '1821', '965', '504', '1165', '1656', '1818', '664']
['Allahabad', '1251', '1686', '1090', '1457', '817', '912', '2216', '650', '1084', '803', '713', '193', '234', '2011', '608', '1155', '1419', '402', '1077', '1364']
['Amritsar', '1356', '2496', '2224', '1849', '1919', '239', '3163', '445', '1892', '1258'

In [218]:
import pandas as pd

# Again Reading Csv file

csv_file = "Road_Distance.csv"
df = pd.read_csv(csv_file)
print(df.head())

  Distance in Kilometres Ahmedabad Bangalore Bhubaneshwar Bombay Calcutta  \
0               Agartala      3305      3824         2286   3593     1863   
1                   Agra       878      1848         1578   1202     1300   
2              Ahmedabad         -      1490         1697    552     2068   
3              Allahabad      1251      1686         1090   1457      817   
4               Amritsar      1356      2496         2224   1849     1919   

  Chandigarh Cochin Delhi Hyderabad  ... Jaipur Kanpur Lucknow Madras Nagpur  \
0       2998   4304  2708      3330  ...   2801   2281    2252   3493   2696   
1        448   2278   200      1246  ...    230    290     369   2048    770   
2       1157   1845   911      1436  ...    648   1168    1247   1821    965   
3        912   2216   650      1084  ...    713    193     234   2011    608   
4        239   3163   445      1892  ...    706    926     939   2688   1416   

  Nasik Panjim Patna Pondicherry  Pune  
0  3365   3507 

In [219]:
def loadRoadDistanceData(file_name):
    distanceData = {}
    with open(file_name, 'r') as data_file:
        fileContent = csv.reader(data_file)
        cityNames = next(fileContent)  # Extract the header row for city names

        rowIndex = 0
        dataRows = list(fileContent)
        while rowIndex < len(dataRows):  # Loop through the rows
            currentRow = dataRows[rowIndex]
            cityA = currentRow[0]
            if cityA not in distanceData:
                distanceData[cityA] = {}

            columnIndex = 1
            while columnIndex < len(currentRow):
                distStr = currentRow[columnIndex]
                if distStr != '-':
                    distValue = int(distStr)

                    cityB = cityNames[columnIndex]

                    if cityB not in distanceData:
                        distanceData[cityB] = {}

                    distanceData[cityA][cityB] = distValue
                    distanceData[cityB][cityA] = distValue  # Update the reverse direction
                columnIndex += 1

            rowIndex += 1

    return distanceData

In [220]:
def UniformCostSearch(file, start, target):
    frontier = [(0, start, [])]  # (TotalCost, CurrentCity, path)
    Exploring = set()

    while frontier:
        frontier.sort()  # Sort by TotalCost
        TotalCost, CurrentCity, path = frontier.pop(0)

        if CurrentCity == target:
            return path + [CurrentCity], TotalCost

        if CurrentCity in Exploring:
            continue

        Exploring.add(CurrentCity)

        NeighBoring = iter(file[CurrentCity].items())
        while True:
            try:
                neighbor, cost = next(NeighBoring)
                if neighbor not in Exploring:
                    new_total_cost = TotalCost + cost
                    new_path = path + [CurrentCity]
                    frontier.append((new_total_cost, neighbor, new_path))
            except StopIteration:
                break

    return None, 0


A* Star admissible

In [308]:
import csv
import heapq
import math

In [325]:
def get_coordinate_distance(file_name):
    data_road_distance = {}
    coords_city = {}

    with open(file_name, 'r') as data_file:
        data_reader = csv.reader(data_file)
        list_cities = next(data_reader)  # Get city names from the header row

        data_row = next(data_reader, None)
        while data_row is not None:
            city_A = data_row[0]
            data_road_distance[city_A] = {}

            # Ignore rows without coordinates
            if data_row[-2] == '-' or data_row[-1] == '-':
                data_row = next(data_reader, None)
                continue

            lat_city = float(data_row[-2])  # Latitude is in the second-to-last column
            lon_city = float(data_row[-1])  # Longitude is in the last column
            coords_city[city_A] = (lat_city, lon_city)

            index = 1
            while index < len(list_cities):
                city_B = list_cities[index]
                str_distance = data_row[index]
                if str_distance != '-':
                    distance_val = int(str_distance)
                    data_road_distance[city_A][city_B] = distance_val
                    if city_B not in data_road_distance:
                        data_road_distance[city_B] = {}
                    data_road_distance[city_B][city_A] = distance_val  # Add the reverse pair (B, A) here
                index += 1

            data_row = next(data_reader, None)
    return coords_city


In [331]:
def search_astar_Admissible(graph, start_node, end_node, coords_city):
    set_open = []
    set_closed = set()
    score_g = {city: float('inf') for city in graph}
    score_g[start_node] = 0
    score_f = {city: float('inf') for city in graph}
    score_f[start_node] = estimate_cost(start_node, end_node, coords_city)

    heapq.heappush(set_open, (score_f[start_node], start_node))
    path_from = {}

    for current_node in iter(lambda: heapq.heappop(set_open)[1] if set_open else None, None):

        if current_node == end_node:
            path_result = rebuild_path(start_node, end_node, path_from)
            distance_travelled = score_g[end_node]
            return path_result, distance_travelled

        set_closed.add(current_node)

        list_neighbors = list(graph[current_node])
        idx = 0
        while idx < len(list_neighbors):
            neighbor_node = list_neighbors[idx]
            if neighbor_node in set_closed:
                idx += 1
                continue

            tentative_score_g = score_g[current_node] + graph[current_node][neighbor_node]

            if tentative_score_g < score_g[neighbor_node]:
                path_from[neighbor_node] = current_node
                score_g[neighbor_node] = tentative_score_g
                score_f[neighbor_node] = score_g[neighbor_node] + estimate_cost(neighbor_node, end_node, coords_city)
                if neighbor_node not in set_open:
                    heapq.heappush(set_open, (score_f[neighbor_node], neighbor_node))
            idx += 1

    return None, 0  # No path found, return a distance of 0


In [332]:
# This A* admissible function
# Function to estimate the cost (using straight-line distance with city coordinates)
def estimate_cost(start_city, end_city, coords_city):
    return 0

In [333]:
# Function to rebuild the path from the A* search
def rebuild_path(start_node, end_node, path_from):
    node_current = end_node
    path_result = [node_current]
    for node_current in iter(lambda: path_from[node_current] if node_current != start_node else None, None):
        path_result.insert(0, node_current)
    return path_result

In [334]:
# Function to calculate distance between two coordinates using the Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    # Earth's radius in kilometers
    earth_radius = 6371

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1

    return delta_lat+delta_lon


In [335]:
import csv
import heapq
import math

In [394]:
def inadmissible_heuristic(city, goal, city_coords):
    return   1000
    
def astar_inadmissible_search(city_map, start_location, end_location, city_coordinates):
    priority_queue = [(0, start_location, [])]  # (cumulative_cost, current_location, route)
    explored_locations = set()

    while priority_queue:
        priority_queue.sort()  # Sort by cumulative_cost
        cumulative_cost, current_location, route = priority_queue.pop(0)

        if current_location == end_location:
            return route + [current_location], cumulative_cost

        if current_location in explored_locations:
            continue

        explored_locations.add(current_location)

        city_connections = list(city_map[current_location].items())
        index = 0
        while index < len(city_connections):
            adjacent_city, travel_cost = city_connections[index]
            if adjacent_city not in explored_locations:
                new_cumulative_cost = cumulative_cost + travel_cost
                heuristic_cost = inadmissible_heuristic(adjacent_city, end_location, city_coordinates)
                new_route = route + [current_location]
                priority_queue.append((new_cumulative_cost + heuristic_cost, adjacent_city, new_route))
            index += 1

    return None, 0 # No path found


In [395]:
# Function to rebuild the path from the A* search
def rebuild_path(start_node, end_node, path_from):
    node_current = end_node
    path_result = [node_current]
    for node_current in iter(lambda: path_from[node_current] if node_current != start_node else None, None):
        path_result.insert(0, node_current)
    return path_result

In [396]:
# Function to calculate distance between two coordinates using the Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    # Earth's radius in kilometers
    earth_radius = 6371

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    a = math.sin(delta_lat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(delta_lon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance_result = earth_radius * c

    return distance_result


In [397]:
file_name = "Road_Distance.csv"
distance_data = loadRoadDistanceData(file_name)
city_coords = get_coordinate_distance(file_path)
while 1:
    print("\nMenu:")
    print("1. Uniform Cost Search")
    print("2. A* admissible")
    print("3. A* inadmissible")
    print("4. Exit")

    enterTheoption = int(input("Enter the option 1,2,3,4 "))

    if enterTheoption == 1:    
        origin_city = input("Enter the origin city: ")
        destination_city = input("Enter the destination city: ")
        
        if origin_city in distance_data and destination_city in distance_data:
            path, total_dist = UniformCostSearch(distance_data, origin_city, destination_city)
            if path:
                print("Shortest path from {} to {}:".format(origin_city, destination_city))
                print(" -> ".join(path))
                print("Total distance: {} kilometers".format(total_dist))
            else:
                print("No path found from {} to {}.".format(origin_city, destination_city))
        else:
            print("Invalid city names. Please verify the city names and try again.")

    elif enterTheoption == 2:
        origin_city = input("Enter the origin city: ")
        destination_city = input("Enter the destination city: ")
        
        if origin_city in distance_data and destination_city in distance_data:
            path, distance_covered = search_astar_Admissible(distance_data, start_city, goal_city, city_coords)
            if path:
                print("Shortest path from {} to {}:".format(origin_city, destination_city))
                print(" -> ".join(path))
                print("Total distance: {} kilometers".format(distance_covered))
            else:
                print("No path found from {} to {}.".format(origin_city, destination_city))
        else:
            print("Invalid city names. Please verify the city names and try again.")
    elif enterTheoption == 3:
        origin_city = input("Enter the origin city: ")
        destination_city = input("Enter the destination city: ")
        
        if origin_city in distance_data and destination_city in distance_data:
            path, distance_covered = search_astar_InAdmissible(distance_data, start_city, goal_city, city_coords)
            if path:
                print("Shortest path from {} to {}:".format(origin_city, destination_city))
                print(" -> ".join(path))
                print("Total distance: {} kilometers".format(distance_covered))
            else:
                print("No path found from {} to {}.".format(origin_city, destination_city))
        else:
            print("Invalid city names. Please verify the city names and try again.")
    elif enterTheoption == 4:
        break
    else:
        print("Invalid option")


Menu:
1. Uniform Cost Search
2. A* admissible
3. A* inadmissible
4. Exit


Enter the option 1,2,3,4  3
Enter the origin city:  Delhi
Enter the destination city:  Pune


Shortest path from Delhi to Pune:
Delhi -> Pune
Total distance: 2417 kilometers

Menu:
1. Uniform Cost Search
2. A* admissible
3. A* inadmissible
4. Exit


KeyboardInterrupt: Interrupted by user

# Compare both the algorithms in terms of performance and explain

Uniform Cost Search (UCS) and A* search are both informed search algorithms that find the shortest path between two nodes in a graph.
However, they both differ in performance performance and optimality.

Uniform Cost Search:
1. UCS is a  of variant of Dijkstra's algorithm and it also finds optimal solutions as this algorithm explores all possible paths in their respective increasing order of their cost.
2. It explores the search space,  that spreads out from the start node to the end node.
3. It will always find the shortest path if one exists. So it is a complete algorithm
4. It can handle graphs with non-uniform costs.
The time complexity in the worst case is exponential in the worst case as it explores all paths.

A Search:
1. A* search algorithm that combines the benefits of uniform cost search to find the best solution(With heuristic function).
2. It uses a heuristic function to estimate the cost from the current node to the goal.
3. A*  algorithm provides the admissible heuristic (never overestimates the actual cost). So, this algorithm is complete
4. It explores the solution better than UCS.

# Explain the heuristics chosen for admissible and non-admissible.
Chosen Heuristic for admissible  is 0 as it is the cost to reach the goal while being optimistic, and this is not the true code this value can be less than or equal to uniform cost search
Here it is lower than non admissible
The Chosen Heuristic for nonadmissile is 1000 as it estimates the cost to reach the goal but it can overestimate the true cost. Most of the time 
nonadmissiblethe  cost is greater than uniform cost search.
Here inaccuracy rate is high